# Traffic Tune - Optimizing Traffic Signals with Reinforcement Learning

## Introduction

Welcome to the Traffic Tune POC notebook. Our project focused on optimizing traffic signal control using reinforcement learning. Traffic congestion is a major problem in urban areas, leading to increased travel times, fuel consumption, and pollution. Traditional traffic signal control systems often struggle to adapt to dynamic traffic conditions, resulting in suboptimal traffic flow.

Traffic Tune is a recommendation system that leverages reinforcement learning to dynamically adjust traffic signals at intersections. By learning from traffic patterns in real-time, Traffic Tune aims to improve traffic flow, reduce congestion, and enhance overall transportation efficiency.

In this POC, we will demonstrate how to train a reinforcement learning agent to optimize traffic signal control in a simulated environment. We will use the SUMO (Simulation of Urban MObility) traffic simulation tool and the Stable Baselines3 library to train a Deep Q-Network (DQN) agent to learn an optimal traffic signal control policy.


# Setup and Installations

In [1]:
import env_manager as env_manager
import ppo_trainer as ppo_trainer

In [ ]:
num_intersection_to_train = 3 # Choose which intersection you want to train

experiment_type = "SingleAgent" # Choose the experiment_type: SingleAgent | MultiAgent

env setup

In [2]:
manager = env_manager.EnvManager("SingleAgentEnvironment", "env_config.json", json_id=f"intersection_{num_intersection_to_train}")
generator = manager.env_generator(f"Nets/intersection_{num_intersection_to_train}/route_xml_path_intersection_{num_intersection_to_train}.txt")
rou , csv = next(generator)
env_kwargs = manager.initialize_env(rou, csv)

print(f" Env creat for intersection_{num_intersection_to_train}", "\n Route path:", rou,"\n Csv path:", csv)

 Env creat for intersection_3 
 Route path: Nets/intersection_3/routes_3/intersection_3_random_easy_1.rou.xml 
 Csv path: Outputs/Training/intersection_3/experiments/intersection_3_random_easy_1_07.21-12:40:25


agent setup

In [3]:
ppo_agent = ppo_trainer.PPOTrainer("ppo_config.json", manager, experiment_type=experiment_type)
ppo_agent.build_config()

2024-07-21 12:40:31,455	INFO worker.py:1771 -- Started a local Ray instance.


agent training

In [ ]:
results = ppo_agent.train()

In [ ]:
print(results.get_dataframe().get("env_runners/episodes_timesteps_total"))
# num_env_runners * num_episode * batch_size = 3600k | 720K

agent prediction

In [ ]:
ppo_agent.evaluate(results=results, kwargs=env_kwargs)

In [ ]:
best = results.get_best_result("env_runners/episode_reward_mean", "max")
# print(best)

In [ ]:
rou , csv = next(generator)
print(rou)
print(csv)
env_kwargs = manager.initialize_env(rou, csv)
print(env_kwargs)

In [ ]:
ppo_agent.config = best.config
ppo_agent.build_config(env_kwargs)

In [ ]:
result_2 = ppo_agent.train()